## 6510405750 ภูมิระพี เสริญวณิชกุล

# SVM

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [10]:
df = pd.read_csv('data/cancer.csv')
df.head(10)

,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,CA level
0,F,Y,Y,Y,Y,14.5,261,2.60,156,1718.0,137.95,172,190,12.2,4
1,F,N,Y,Y,N,1.1,302,4.14,54,7394.8,113.52,88,221,10.6,3
2,M,N,N,N,S,1.4,176,3.48,210,516.0,96.10,55,151,12.0,4
3,F,N,Y,Y,S,1.8,244,2.54,64,6121.8,60.63,92,183,10.3,4
4,F,N,Y,Y,N,3.4,279,3.53,143,671.0,113.15,72,136,10.9,3
5,F,N,Y,N,N,0.8,248,3.98,50,944.0,93.00,63,200,11.0,3
6,F,N,Y,N,N,1.0,322,4.09,52,824.0,60.45,213,204,9.7,3
7,F,N,N,N,N,0.3,280,4.00,52,4651.2,28.38,189,373,11.0,3
8,F,N,N,Y,N,3.2,562,3.08,79,2276.0,144.15,88,251,11.0,2
9,F,Y,N,Y,Y,12.6,200,2.74,140,918.0,147.25,143,302,11.5,4


In [11]:
df['Sex'] = df['Sex'].map({'M': 1, 'F': 0})
df['Ascites'] = df['Ascites'].map({'Y': 1, 'N': 0, 'S': 2})
df['Hepatomegaly'] = df['Hepatomegaly'].map({'Y': 1, 'N': 0, 'S': 2})
df['Spiders'] = df['Spiders'].map({'Y': 1, 'N': 0, 'S': 2})
df['Edema'] = df['Edema'].map({'Y': 1, 'N': 0, 'S': 2})

X = df.drop(columns=['CA level']) # feature
y = df['CA level'] # class

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3, shuffle=True) # แบ่งข้อมูลเป็น train และ test

In [13]:
# scaler ทำเพื่อปรับข้อมูลให้อยู่ในช่วง 0-1 และทำให้การคำนวณเร็วขึ้น
scaler = StandardScaler() # ปรับข้อมูลให้อยู่ในช่วง 0-1
X_train = scaler.fit_transform(X_train) # ปรับข้อมูล train
X_test = scaler.transform(X_test) # ปรับข้อมูล test

In [14]:
grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1, 3, 5, 7],
    'gamma': [0.01, 1]
}

svm_cv = GridSearchCV(SVC(), grid, cv=5)
svm_cv.fit(X_train, y_train)
print("Best Parameters:", svm_cv.best_params_) 
print("Train Score:", svm_cv.best_score_)
print("Test Score:", svm_cv.score(X_test, y_test))

Best Parameters: {'C': 10, 'degree': 1, 'gamma': 0.01, 'kernel': 'sigmoid'}
Train Score: 0.5072938689217759
Test Score: 0.5483870967741935


In [15]:
# สร้าง model และ train ด้วย OneVsOneClassifier และ LinearSVC
clf = OneVsOneClassifier(LinearSVC(dual="auto", random_state=0)).fit(X_train, y_train)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("==================================================")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("==================================================")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.5483870967741935
Confusion Matrix:
[[ 0  2  0  0]
 [ 0  5 10  2]
 [ 0  4 23  7]
 [ 1  0 16 23]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.45      0.29      0.36        17
           3       0.47      0.68      0.55        34
           4       0.72      0.57      0.64        40

    accuracy                           0.55        93
   macro avg       0.41      0.39      0.39        93
weighted avg       0.56      0.55      0.54        93

